In [8]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import csv
import gmaps

# Google API Key
from config import gkey

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

%matplotlib inline

In [9]:
# Import CSV
g_lookup_pd = pd.read_csv("Resources/Data Filtration.csv")

In [10]:
# Rename columns for readability
g_lookup_pd_rn = g_lookup_pd.rename(columns={"restaurant_name": "Restaurant",
                                             "inspection_type": "Inspection Type",
                                             "inspection_date": "Inspection Date",
                                             "inspection_score": "Inspection Score",
                                             "street_number": "Street Number",
                                             "street_name": "Street Name",
                                             "street_unit": "Street Unit",
                                             "zip_code": "Zip",
                                             "street_address": "Street Address",
                                             "inspection_month": "Inspection Month",
                                             "inspection_year": "Inspection Year",
                                                })

g_lookup_pd_rn["Lat"] = ""
g_lookup_pd_rn["Long"] = ""

g_lookup_pd_rn

,Restaurant,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long
0,2204,Routine,6/28/2019,95,2204,ELM,NaN,75201,2204 ELM ST,6/1/2019,FY2019,,
1,3606,Routine,12/30/2019,97,3606,GREENVILLE,#B,75206,3606 GREENVILLE AVE #B,12/1/2019,FY2020,,
2,9999,Routine,12/20/2017,98,1011,WALTON WALKER,NaN,75211,1011 S WALTON WALKER BLVD,12/1/2017,FY2018,,
3,13209,Routine,8/7/2019,96,13209,C F HAWN,NaN,75253,13209 C F HAWN FRWY,8/1/2019,FY2019,,
4,44023,Routine,1/10/2020,81,4024,ROSS,NaN,75204,4024 ROSS AVE,1/1/2020,FY2020,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6763,ZOE'S KITCHEN,Routine,9/23/2019,89,6025,ROYAL,104,75230,6025 ROYAL LN STE 104,9/1/2019,FY2019,,
6764,ZOE'S KITCHEN,Routine,8/14/2019,94,5710,LOVERS,#108,75209,5710 W LOVERS LN #108,8/1/2019,FY2019,,
6765,ZOE'S KITCHEN #0031,Routine,11/6/2019,88,2009,ABRAMS,NaN,75214,2009 ABRAMS RD,11/1/2019,FY2020,,
6766,ZRG CONCEPTS,Routine,9/16/2019,96,2505,FITZHUGH,NaN,75204,2505 N FITZHUGH AVE,9/1/2019,FY2019,,


In [12]:
# Organize the columns so they are in a more logical order

#g_lookup_pd_org = g_lookup_pd_rn[[
    #"Restaurant", "Inspection Date", "Inspection Score", "Street Address", "Zip", "Lat", "Long"]]

#g_lookup_pd_org

,Restaurant,Inspection Date,Inspection Score,Street Address,Zip,Lat,Long
0,2204,6/28/2019,95,2204 ELM ST,75201,,
1,3606,12/30/2019,97,3606 GREENVILLE AVE #B,75206,,
2,9999,12/20/2017,98,1011 S WALTON WALKER BLVD,75211,,
3,13209,8/7/2019,96,13209 C F HAWN FRWY,75253,,
4,44023,1/10/2020,81,4024 ROSS AVE,75204,,
...,...,...,...,...,...,...,...
6763,ZOE'S KITCHEN,9/23/2019,89,6025 ROYAL LN STE 104,75230,,
6764,ZOE'S KITCHEN,8/14/2019,94,5710 W LOVERS LN #108,75209,,
6765,ZOE'S KITCHEN #0031,11/6/2019,88,2009 ABRAMS RD,75214,,
6766,ZRG CONCEPTS,9/16/2019,96,2505 N FITZHUGH AVE,75204,,


In [22]:
# Apply filters for testing (set limit)

# Create a copy of re-organized Data
g_lookup_pd_filter = g_lookup_pd_rn.copy()

# Filter by Inspection Score
g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Inspection Score'] == 100]

# Filter by Zip
g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Zip'] == "75237"]

# Drop null values. In this case, we're keeping it since we're looking up by street address and zip.
# Otherwise, it will drop all data missing a street unit.
#g_lookup_pd_filter = g_lookup_pd_filter.dropna()
g_lookup_pd_filter = g_lookup_pd_filter.reset_index()

g_lookup_pd_filter

,index,Restaurant,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long
0,40,"16096 TRINITY BASIN PREPARATORY, INC",Routine,10/8/2019,100,2524,LEDBETTER,BU4,75237,2524 W LEDBETTER DR BU4,10/1/2019,FY2020,,
1,277,ACCENT FOOD SERVICES @ PALLET LOGISTICS,Routine,9/23/2019,100,4100,PLATINUM,NaN,75237,4100 W PLATINUM WAY,9/1/2019,FY2019,,
2,3942,MICKLE CHICKEN,Routine,10/30/2019,100,3203,CAMP WISDOM,NaN,75237,3203 W CAMP WISDOM RD,10/1/2019,FY2020,,
3,6507,WELLMED SENIOR ACTIVITY CENTER,Routine,9/9/2019,100,3107,CAMP WISDOM,#175,75237,3107 W CAMP WISDOM RD #175,9/1/2019,FY2019,,


In [ ]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the g_lookup_pd_org and run a lat/long search for each city
for index, row in g_lookup_pd_filter.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        street = row['Street Address']
        zip_code = row['Zip']

        # update address key value
        params['address'] = f"{street},{zip_code}"

        # make request
        cities_lat_lng = requests.get(base_url, params=params)

        # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key


        # convert to json
        cities_lat_lng = cities_lat_lng.json()


        
        g_lookup_pd_filter.loc[index, "G.Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        g.lat = g_lookup_pd_filter.loc[index, "G.Lat"]
        
        g_lookup_pd_filter.loc[index, "G.Long"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
        g.long = g_lookup_pd_filter.loc[index, "G.Long"]
        
        if g.lat == "":
            g.lat == "NaN"

    except:
        print("Not found. Keeping calm and carrying on.")

        
        pass
    


In [ ]:
remove_empty_lat_long = g_lookup_pd_filter.copy()

remove_empty_lat_long.replace.dropna(how='all')

remove_empty_lat_long


In [ ]:
# Organize the columns so they are in a more logical order
drop_old_lat_long = g_lookup_pd_filter.copy()
drop_old_lat_long = drop_old_lat_long[[
    "Restaurant", "Inspection Date", "Inspection Score", "Street Address", "Zip", "G.Lat", "G.Long"]]

# Rename G.Lat and G.Long
drop_old_lat_long_rn = drop_old_lat_long.rename(columns={"G.Lat": "Lat",
                                                        "G.Long": "Long"})
drop_old_lat_long_rn

In [ ]:
# Save as CSV
g_lookup_pd_filter.to_csv("Google Lat Long Error.csv")

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

In [ ]:
# Store 'Lat' and 'Lng' into  locations 
locations = g_lookup_pd_filter[["G.Lat", "G.Lng"]]
inspection_score = g_lookup_pd_filter["Inspection Score"].astype(int)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights=inspection_score, 
                                 dissipating=False, max_intensity=80,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig